In [4]:
%reset -f
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.examples.tutorials.mnist import input_data

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 還是用mnist資料集來做例子
mnist = input_data.read_data_sets("MNIST_data", one_hot=False)

# mnist輸入資料的維度
input_dim = 784
# 隱藏層神經元數量
num_hidden = 200
batch_size = 128
test_size = 128

# 定義X佔位符，用來接收輸入資料
X = tf.placeholder(tf.float32, [None, input_dim])
# scale為噪聲水平
scale_value = 0.01
x_input = X + scale_value * tf.random_normal((input_dim,))
# Xavier均勻初始化,num_in是這層輸入節點的數量，num_out是這層輸出節點的數量。
w_init_max = 1.0 * np.sqrt(6.0 / (input_dim + num_hidden))
# W_init的shape=[784,1024]
w_init = tf.random_uniform(shape=[input_dim, num_hidden], minval=-w_init_max, maxval=w_init_max)
# 編碼器w和b權重
w_e = tf.Variable(w_init, name='w_encoder')
b_e = tf.Variable(tf.zeros([num_hidden]), name='b_encoder')
# 解碼器w和b權重，解碼器的w是編碼器w的轉置
w_d = tf.transpose(w_e, name='w_decoder')
b_d = tf.Variable(tf.zeros([input_dim]), name='b_decoder')


# 定義自動編碼器模型
# 輸入x的shape=[batch_size,784]，w_encoder的shape=[784,1024],w_decoder的shape=[1024,784]
def model(x, w_encoder, b_encoder, w_decoder, b_decoder):
	y = tf.nn.sigmoid(tf.add(tf.matmul(x, w_encoder), b_encoder))
	z = tf.nn.sigmoid(tf.add(tf.matmul(y, w_decoder), b_decoder))
	return z


# 得到預測值
X_pred = model(x_input, w_e, b_e, w_d, b_d)

# 定義損失函式為平方和均值,並且加入L2正則化項，這樣要快很多
# 注意計算的是原始資料x與pred之間的誤差，而不是加了噪聲以後的資料x_input
cost = tf.reduce_mean(tf.square(X - X_pred))
train_op = tf.train.AdamOptimizer(0.01).minimize(cost)

saver = tf.train.Saver()
# 建立儲存模型的路徑
if not os.path.exists("./save_model/"):
	os.mkdir("./save_model")

with tf.Session() as sess:
	sess.run(tf.global_variables_initializer())
	if os.path.exists("./save_model/checkpoint"):
		# 判斷模型是否存在，如果存在則從模型中恢復變數
		saver.restore(sess, tf.train.latest_checkpoint('./save_model/'))
	step = 0
	while True:
		batch_train_x_data, batch_train_y_data = mnist.train.next_batch(batch_size)
		loss, _ = sess.run([cost, train_op], feed_dict={X: batch_train_x_data})
		if step % 50 == 0:
			print("iteration:{:>06d} loss:{}".format(step, loss))
			saver.save(sess, "./save_model/train_model", global_step=step)
		if loss < 0.003:
			break
		if step > 10000:
			break
		step += 1
	print("Training finished!")
	# 訓練好模型後，我們用模型來編碼壓縮後再解碼，看看和原圖相比的效果
	test_x_data, test_y_data = mnist.test.next_batch(test_size)
	test_x_decode = sess.run(X_pred, feed_dict={X: test_x_data})
	# 將解碼後的圖片與原始圖片做比較，這裡只比較10張
	# 把影象的幕布分成2行10列，figsize=(10, 2), dpi=100即建立一張1000X200畫素的圖
	f, a = plt.subplots(2, 10, figsize=(10, 2), dpi=100)
	for i in range(10):
		a[0][i].imshow(np.reshape(test_x_data[i], (28, 28)), cmap=plt.get_cmap('gray'))
		a[1][i].imshow(np.reshape(test_x_decode[i], (28, 28)), cmap=plt.get_cmap('gray'))
	plt.show()
	plt.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from ./save_model/train_model-9350


NotFoundError: Key b_decoder_1 not found in checkpoint
	 [[Node: save_3/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_3/Const_0_0, save_3/RestoreV2/tensor_names, save_3/RestoreV2/shape_and_slices)]]
	 [[Node: save_3/RestoreV2/_29 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_34_save_3/RestoreV2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'save_3/RestoreV2', defined at:
  File "/home/joshhu/anaconda3/envs/tensorflow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/joshhu/anaconda3/envs/tensorflow/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home